In [1]:
import pandas as pd
import os
import re
import time
import numpy as np
import statistics
from natsort import natsorted, ns
import random
from sklearn.preprocessing import StandardScaler 

from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from xgboost import XGBClassifier

In [2]:
def spatial_two (Node_class, Adj,n):
    F_vec=[]
    for i in range(n):
        print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if Adj[i][j]==1 and i!=j:
                list_out.append(Adj[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(Adj[k][k])
            if Adj[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if Adj[k][j]==1 and k!=j:
                        S_nbd_in.append(Adj[k][k])

        for d in Node_class:
            count=0
            count_in=0
            for k in range(len(list_out)):
                if list_out[k]==d:
                    count=count+1
            node_F.append(count)
            for k in range(len(list_In)):
                if list_In[k]==d:
                    count_in=count_in+1
            node_F.append(count_in)
        for d in Node_class:
            count_S_out=0
            count_S_in=0
            for k in range(len(S_nbd_out)):
                if S_nbd_out[k]==d:
                    count_S_out=count_S_out+1
            node_F.append(count_S_out)
            for k in range(len(S_nbd_in)):
                if S_nbd_in[k]==d:
                    count_S_in=count_S_in+1
            node_F.append(count_S_in)
        F_vec.append(node_F)
    return F_vec

In [3]:
import pandas as pd
import numpy as np
# read the dataset from the link, extract and make a dataframe
import requests, zipfile, io
r = requests.get('https://linqs-data.soe.ucsc.edu/public/datasets/cora/cora.zip', stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names = ["ID"]+ feature_names + ["class"]
node_data = pd.read_csv("cora/cora.content", sep='\t', header=None, names=column_names)
node_data.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


In [4]:
DataS=node_data.replace({'class': {'Case_Based':0 , 'Genetic_Algorithms': 1,'Neural_Networks':2,'Probabilistic_Methods':3 , 'Reinforcement_Learning': 4,'Rule_Learning':5,'Theory':6}})
# Case_Based==0
#		Genetic_Algorithms==1
#		Neural_Networks==2
#		Probabilistic_Methods==3
#		Reinforcement_Learning==4
#		Rule_Learning==5
#		Theory==6
DataS.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [5]:
Data=DataS.sort_values(by=['ID'],ascending=True)
Data.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
163,35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
168,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
552,114,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,4
1459,117,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
554,128,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [6]:
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class', 'ID'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[f'w_{i}']) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[f'w_{i}'].to_numpy()).count(1) >= int(len(df[f'w_{i}'].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
IdS=list(DataS['ID'].to_numpy())
Id=list(Data['ID'].to_numpy())

In [7]:
Fec=[]
for i in range(2708):
    vec=[]
    f=Data.loc[IdS.index(Id[i]), feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    count6=0
    for j in range(1433):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
        if f[j]==1 and basis[6][j]==1:
            count6=count6+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    vec.append(count6)
    #print(f)
    f.clear()
    Fec.append(vec)

In [8]:
SFec=[]
for i in range(2708):
    Svec=[]
    f=Data.loc[IdS.index(Id[i]), feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    count6=0
    for j in range(1433):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
        if f[j]==1 and sel_basis[6][j]==1:
            count6=count6+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    Svec.append(count6)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [9]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(6):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4,S_5,S_6
0,1,7,3,2,3,2,1
1,6,5,3,4,6,7,4
2,1,3,1,1,4,3,1
3,5,4,4,4,8,2,5
4,5,3,3,2,8,2,3


In [11]:
Fe=[]
for i in range(7):
    Fe.append("{}".format(i))
y =np.array(Fec)
data1 = pd.DataFrame(y, columns =Fe)
data1.head()

,0,1,2,3,4,5,6
0,16,20,20,16,17,12,14
1,16,18,18,17,15,16,15
2,10,11,12,10,12,7,8
3,13,17,19,18,20,14,18
4,17,17,19,17,20,15,17


In [12]:
p_data = open("cora/cora.cites")


# Create a list from the data
myls=list(p_data)

# Remove the new line symbole from the list
mylist = [line.rstrip('\n') for line in myls]
#print(mylist)
#print(graph_ind)
#print(graph_level)

p_data.close()
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
#print(node_ID)
n=len(node_ID)
print(n)

2708


In [13]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
D_only=[]
S_only=[]
Both=[]

In [15]:
p=random.sample(range(2708), 1000)
#print(p)
q=[]
for i in range(n):
    if i not in p:
        q.append(i)
#print(q)
class_60=np.array(catagories)
for d in p:
    class_60[d]=7
print(class_60)
for i in range(n):
    A[i][i]=class_60[i]

[1 7 4 ... 5 7 7]


In [17]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]
    p=random.sample(range(2708), 1000)
    #print(p)
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    #print(q)

    #LABLE THE UNKNOWN/TEST NODE AS 6 AND PLACE CLASS TO THE DIAGONAL OF ADJACENCY MATRIX

    class_60=np.array(catagories)
    for d in p:
        class_60[d]=7
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]

    Node_class=[0,1,2,3,4,5,6]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)

    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=len(F_vec[0])
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    l=7
    for i in range(l):
        feature.append("S_{}".format(i))
    for i in range(l):
        feature.append("{}".format(i))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    l=7
    for i in range(l):
        feature.append("S_{}".format(i))
    for i in range(l):
        feature.append("{}".format(i))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

Processing file 2707 (100%)

In [18]:
mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial Only
Mean: 80.24000000000001
Standard Deviation: 1.561480778691244
Domain only
Mean: 83.0
Standard Deviation: 1.2806248474865676
Accuracy using both
Mean: 87.27
Standard Deviation: 0.9522488003551269
